# **NLP Intent Parser for Industrial Technician Queries**

A modular pipeline consisting of:
1. Topic Router (LDA, SVM, Mini-BERT)
2. Intent + Target + Parameter Token Classifier (DistilBERT, BiLSTM, LSTM)
3. Context Resolver for domain-aware refinement

This notebook demonstrates preprocessing, embeddings, token labeling, 
three different modeling strategies, evaluation, and comparison.


### **1. Import and Setup**

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pandas numpy scikit-learn nltk torch seaborn matplotlib transformers tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizer




from transformers import AutoTokenizer, AutoModelForTokenClassification

###  **2. Load Technician Query Dataset**

**Why We Generated the Dataset Ourselves**

There isn’t any publicly available dataset that captures "technician-style" micro-grid instructions with the level of structure we need (intent, target, parameter, modifier, conditions). Real industrial datasets are either private, messy, and rarely come with clean labels or ones we can make sense of. Since our goal here is to benchmark different NLP models, not to clean handwritten maintenance logs, synthetic data gives us full control over the balance, coverage, and consistency.

It lets us shape the exact problem in the manner that we want to model, and it’s standard practice during early prototyping before fine-tuning on real operational data later.

In [ ]:
df = pd.read_csv('./data/solar_ds.csv')    

### **3. Data Exploration (EDA)**

**The first step is to confirm formatting and make sure all columns loaded correctly.**

*Our EDA focuses on validating distribution, coverage, and linguistic variety across intents, targets, and parameters. Since the dataset is synthetic, the goal isn’t noise inspection but ensuring balance, realism, and sufficient diversity to train and compare NLP models reliably.*

In [ ]:
df.head()

In [ ]:
df.sample(5)

In [ ]:
df.info()

### **4. Preprocessing Functions**

*Even though the dataset is synthetic and noise-free, preprocessing is still required to prepare the data for deep learning models. This includes tokenization, padding/truncation to a fixed sequence length, and label encoding. We skip stopword removal, lemmatization, and other cleaning steps because our goal is to preserve the natural language variation that helps the model learn intent patterns.*

### **5. Topic Modeling Module**

#### **5.1 TF-IDF + SVM Baseline**

#### **5.2 LDA Topic Modeling (Unsupervised)**

#### **5.3 MiniBERT Topic Classifier (Supervised)**

### **6. Compare Topic Models**

### **7. Token Classification Dataset Preparation**

### **8. Model 1: DistilBERT Token Classifier**

### **9. Model 2: BiLSTM Token Classifier**

### **10. Model 3: Simple LSTM Tagger**

### **11. Training Loops (All Models)**

### **12. Evaluation: Intent, Target, Parameter Extraction**

### **13. Context Resolver Logic**

### **14. End-to-End Pipeline Demonstration**

### **15. Model Comparison Summary (The MLE Signal)**

### **16. Conclusions & Future Work**

Include:

- integrate with GridGuard

- replace LDA with BERTopic

- build your own transformer from scratch (future project)

- deploy as microservice